# AIA Systems Lab 
### Submission for MBRDI CEA


In [12]:
from collections import OrderedDict
import xml.etree.ElementTree as ET
from collections import OrderedDict
import numpy as np
from numpy import linalg as LA
import math

#fixing pole length value (as extracted from .skp file)
polelength = 1.6
polewidth = 0.1

#fixing the LIDAR's specs
horzmax = 72.5
uemax = 15
lemin = -25

#hard coding the position of the car/LIDAR and also LIDAR's range. If a path is to be given, this can be updated accordingly.
car_posx = 20
car_posy = 13.375
car_posz = 0.5
lidar_range = 200
carpos = np.asarray([car_posx,car_posy,car_posz])

Obtain signal data from XODR file

In [13]:
def readXODR_sort(xodr, carpos):
    #This function is to read the XODR file, extract signals and thereafter sort it according to distance
    #It returns a dictionary containing all signs and relevant data
    yoffsetglobal=13.375
    tree = ET.parse(xodr)
    root = tree.getroot()
    signal1 = {}
    i = 0

    for roadno,road in enumerate(root.iter('road')):
    
        for pv in road.iter('planView'):
            for geometry in pv.iter('geometry'):
                xoffset = float(geometry.attrib['x'])
                yoffset = float(geometry.attrib['y'])
                break
            break
        
        for signal in road.iter('signal'):
            xg = float(signal.attrib['s'])+xoffset
            yg = float(signal.attrib['t'])+yoffset-yoffsetglobal
            zg = 0.0
    
            signal1[i] = {'roadno': roadno, 'xglobal': xg,'yglobal': yg,'zglobal': zg,'height': float(signal.attrib['height']),'dist': LA.norm(carpos-np.asarray([xg,yg,zg])),'name':signal.attrib['name'],'carpos':carpos}
            i+=1

    return signal1

In [14]:
def findangle(v1, v2):
    #This function is to find angles between two vectors
    cosang = np.dot(v1, v2)
    sinang = LA.norm(np.cross(v1, v2))
    return (180/math.pi)*np.arctan2(sinang, cosang)

In [15]:
def signaldatabase(name):
    #This function is used to segregate the signals in terms of shape. Currently hard-coded but can be made dynamic.
    #returns the parameters of the signal
    
    polelength = 1.6
    parameters = {}
    
    if(name == 'Sg274Hoechstgeschw30_03.flt' or name == 'Sg274Hoechstgeschw50_03.flt' or name == 'Sg274Hoechstgeschw50_03.flt'):
        parameters = {'name': 'speedsign', 'shape':'circle', 'pole':polelength}
        
    elif(name=="Sg451.flt"):
        parameters = {'name': 'nearingexit', 'shape': 'rectangle', 'width':0.76, 'pole':polelength}
        
    elif(name=="Sg332Ausfahrttafel.flt"):
        parameters = {'name':'exit', 'shape': 'square', 'pole':polelength}
        
    elif(name == 'Sg331Kraftfahrstr03.flt'):
        parameters = {'name':'maintainlane', 'shape':'square', 'pole': polelength}
        
    else:
        parameters = {'name':'exit', 'shape': 'square', 'pole':polelength}
    
    return parameters

This function sets bounding boxes. It checks angles to see if it is in FOV and distances.

In [16]:
def boundingbox(signalsinrange,carposition):
     
    signal1 = signalsinrange
    
    a = np.zeros([len(signal1.keys()),7])
    temparr={}
    for idx,signal in enumerate(signalsinrange):
        a[idx][0] = signalsinrange[idx]['dist']
        a[idx][1] = signalsinrange[idx]['xglobal']
        a[idx][2] = signalsinrange[idx]['yglobal']
        a[idx][3] = signalsinrange[idx]['zglobal']
        a[idx][4] = signalsinrange[idx]['height']
        temparr = signaldatabase(signal1[idx]['name'])
        
        if (temparr['shape']=='rectangle'):
            a[idx][5] = temparr['width']
            a[idx][6] = 0
            
        elif(temparr['shape']=='circle'):
            a[idx][5] = a[idx][4]
            a[idx][6] = 1
                    
        else:
            a[idx][5] = a[idx][4]
            a[idx][6] = 0
            
    
                    
    ind=np.argsort(a[:,0])
    sortedsignals=a[ind]
    infov = {}
        
    i=0
    
    for idx in range(0,sortedsignals.shape[0]):
        if(sortedsignals[idx][0]>200):
            break     
        if(sortedsignals[idx][2] <= 0):
            
            yordi = sortedsignals[idx][2] + (sortedsignals[idx][5]/2)
            yordo = sortedsignals[idx][2] - (sortedsignals[idx][5]/2)
            yordipole = sortedsignals[idx][2] + polewidth/2
            yordopole = sortedsignals[idx][2] - polewidth/2
            
        else:
            
            yordi = sortedsignals[idx][2] - (sortedsignals[idx][5]/2)
            yordo = sortedsignals[idx][2] + (sortedsignals[idx][5]/2)
            yordipole = sortedsignals[idx][2] - polewidth/2
            yordopole = sortedsignals[idx][2] + polewidth/2
        
        zordh = sortedsignals[idx][3] + sortedsignals[idx][4] + polelength
        zordl = sortedsignals[idx][3] + polelength
        zordhpole = zordl
        zordlpole = sortedsignals[idx][3]
    
        inneredge = [sortedsignals[idx][1],yordi]
        outeredge = [sortedsignals[idx][1],yordo]
        upperedge = [sortedsignals[idx][2],zordh]
        loweredge = [sortedsignals[idx][2],zordl]
        
        inneredgepole = [sortedsignals[idx][1],yordipole]
        outeredgepole = [sortedsignals[idx][1],yordopole]
        upperedgepole = [sortedsignals[idx][2],zordhpole]
        loweredgepole = [sortedsignals[idx][2],zordlpole]
        
        if(yordi < carposition[1]):
            angleie = -(90-findangle([carposition[0],carposition[1]],inneredge))
            angleiepole = -(90-findangle([carposition[0],carposition[1]],inneredgepole))
        else:
            angleie = (90-findangle([carposition[0],carposition[1]],inneredge))
            angleiepole = (90-findangle([carposition[0],carposition[1]],inneredgepole))
        if(yordo < carposition[1]):
            angleoe = -(90-findangle(outeredge,[carposition[0],carposition[1]]))
        else:
            angleoe = (90-findangle(outeredge,[carposition[0],carposition[1]]))
            
        if(yordopole<carposition[1]):
            angleoepole = -(90-findangle(outeredgepole,[carposition[0],carposition[1]]))
        else:
            angleoepole = (90-findangle(outeredgepole,[carposition[0],carposition[1]]))
        
        if((sortedsignals[idx][2]<=0 and carposition[2]<=0) or (sortedsignals[idx][2]>=0 and carposition[2]>=0)):
            if(carposition[2]<=zordh):
                angleue = findangle(upperedge,[carposition[1],carposition[2]])
            else:
                angleue = -findangle(upperedge,[carposition[1],carposition[2]])
            if(carposition[2]<=zordl):
                anglele = findangle(loweredge,[carposition[1],carposition[2]])
            else:
                anglele = -findangle(loweredge,[carposition[1],carposition[2]])
            if(carposition[2]<=zordlpole):
                anglelepole = findangle(loweredgepole,[carposition[1],carposition[2]])
            else:
                anglelepole = -findangle(loweredgepole,[carposition[1],carposition[2]])
                
                
        else:
            if(carposition[2]<=zordh):
                angleue = (180-findangle(upperedge,[carposition[1],carposition[2]]))
            else:
                angleue = -(180-findangle(upperedge,[carposition[1],carposition[2]]))
            if(carposition[2]<=zordl):
                anglele = (180-findangle(loweredge,[carposition[1],carposition[2]]))
            else:
                anglele = -(180-findangle(loweredge,[carposition[1],carposition[2]]))
            if(carposition[2]<=zordlpole):
                anglelepole = (180-findangle(loweredgepole,[carposition[1],carposition[2]]))
            else:
                anglelepole = -(180-findangle(loweredgepole,[carposition[1],carposition[2]]))
                
        
        if(abs(angleie)>abs(angleoe)):
            tmp = angleoe
            angleoe = angleie
            angleie = tmp
            
        if(anglele>angleue):
            tmp = anglele
            anglele = angleue
            angleue = tmp
            
        angleuepole = anglele
            
        if(abs(angleie) <= horzmax):
            
            bboxie = angleie
            bboxiepole = angleiepole
            
            if(angleoe < 0):
                bboxoe = max((-horzmax),angleoe)
            else:
                bboxoe = min(horzmax,angleoe)
            if(angleoepole<0):
                bboxoepole = max((-horzmax),angleoepole)
            else:
                bboxoepole = min(horzmax,angleoepole)
            if(angleue<0):
                bboxue = max(lemin,angleue)
            else:
                bboxue = min(uemax,angleue)
                
                           
            bboxle = max(lemin,anglele)
            bboxlepole = max(lemin,anglelepole)
            bboxuepole = bboxle
            
            infov[i] = {'dist': sortedsignals[idx][0], 'xord':sortedsignals[idx][1], 'bboxie': bboxie, 'bboxoe': bboxoe, 'bboxue':bboxue, 'yordi':yordi, 'yordo':yordo,'zordh':zordh,'zordl':zordl, 'bboxle': bboxle, 'height': sortedsignals[idx][4], 'width':sortedsignals[idx][5], 'circle': sortedsignals[idx][6],'type':'signal','xglobal': a[idx][1],'yglobal': a[idx][2],'zglobal':a[idx][3]}
            i+=1
                
            infov[i] = {'dist':sortedsignals[idx][0],'xord':sortedsignals[idx][1],'bboxie':bboxiepole,'bboxoe':bboxoepole, 'bboxue':bboxuepole, 'bboxle':bboxlepole, 'yordi':yordipole, 'yordo':yordopole, 'zordh':zordhpole,'zordl':zordlpole, 'height':polelength, 'width':polewidth, 'circle':0,'type':'pole'}
            i+=1    
            
    return infov
vs = readXODR_sort('road_specification_v3.xodr',carpos )   
infov = boundingbox(vs,carpos)

This function samples the bounding boxes and generates xyz cloud point data

In [17]:
def sampler(angleY,angleZ, pointsY, pointsZ, visiblesignalRange):
    thetay = angleY/pointsY
    thetaz = angleZ/pointsZ
    print(thetaz)
    print(thetay)
    
    for idx, signal in enumerate(visiblesignalRange):
        
        d = visiblesignalRange[idx]['dist']

        ie = visiblesignalRange[idx]['yordi']
        oe = visiblesignalRange[idx]['yordo']
        ue = visiblesignalRange[idx]['zordh']
        le = visiblesignalRange[idx]['zordl']
        
        iet = visiblesignalRange[idx]['bboxie']
        oet = visiblesignalRange[idx]['bboxoe']
        uet = visiblesignalRange[idx]['bboxue']
        let = visiblesignalRange[idx]['bboxle']
        
        ay = [] #list containing y points
        az = [] #list containing z points
        

        diffy = abs(oet-iet)
        samplesy = diffy/thetay
        stepy = abs(ie-oe)/samplesy
        
        diffz = abs(uet-let)
        samplesz = diffz/thetaz
        stepz = abs(ue-le)/samplesz
        
        if (visiblesignalRange[idx]['circle'] == 1):
            for n in np.arange((oe), ie, stepy):
                for m in np.arange(le, ue, stepz):
                    if (abs((n-visiblesignalRange[idx]['yglobal'])**2 + (m-visiblesignalRange[idx]['zglobal'])**2)<=(visiblesignalRange[idx]['width']/2)**2):
                        ay.append(n)

            for m in np.arange(le, ue, stepz):
                az.append(m)
        
        else:
            for n in np.arange((oe), ie, stepy):
                ay.append(n)
            for m in np.arange((le),ue,stepz):
                az.append(m)
                
      
        visiblesignalRange[idx]['yPoint'] = ay
        visiblesignalRange[idx]['zPoint'] = az
        
               
        

In [18]:
sampler(145,40,100000,64,infov)

0.625
0.00145


This generates the cloud point d

In [19]:
x = []
x.append(infov[0]['xord'])
y = np.asarray(infov[0]['yPoint'])
z = np.asarray(infov[0]['zPoint'])
print(len(y))
print(len(z))
yp = np.asarray(infov[1]['yPoint'])
zp = np.asarray(infov[1]['zPoint'])
print(len(yp))
print(len(zp))

dumpdict = {}
Point_Cloud = []
i = 0
for j in range(0,z.shape[0]):
    for k in range(0, y.shape[0]):
        dumpdict[i] = {"Pos_x":-(y[k]),"Pos_y":x[0],"Pos_z":z[j],"Valid_Flag":"True"} 
        Point_Cloud.append(dumpdict[i])
        i+=1



for j in range(0,zp.shape[0]):
    for k in range(0, yp.shape[0]):
        dumpdict[i] = {"Pos_x":-(yp[k]),"Pos_y":x[0],"Pos_z":zp[j],"Valid_Flag":"True"} 
        Point_Cloud.append(dumpdict[i])
        i+=1

0
3
77
13


In [20]:
import os
import time
import json

jsonformat = {'timestamp':[{"VehiclePosition":[{"Pos_x":car_posy, "Pos_y":car_posx}],"Lidar":[{"PointCloud": Point_Cloud}]}]}

fname = 'Output1.json'

with open(fname, mode='w') as f:
    f.write(json.dumps(jsonformat, indent=2))